# Spam classifier

Explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Exploratory Data analysis
What percentage of the documents in `spam_data` are spam?



In [3]:
def percentage_spam():
    s=spam_data['target'].value_counts()[1]
    t=spam_data.shape[0]
    return s/t*100

In [4]:
percentage_spam()

13.406317300789663

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def Count_vect():
    
    vocabulary = CountVectorizer().fit(X_train).vocabulary_
    vocabulary = [x for x in vocabulary.keys()]
    len_vocabulary = [len(x) for x in vocabulary]
    return vocabulary[np.argmax(len_vocabulary)]

In [6]:
Count_vect()

'com1win150ppmx3age16subscription'

Fitting a basic model

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def MNB():
    
    vect=CountVectorizer().fit(X_train)
    X_train_vect=vect.transform(X_train)
    
    clf=MultinomialNB(alpha=.1)
    clf=clf.fit(X_train_vect,y_train)
    
    X_test_vect=vect.transform(X_test)
    pre=clf.predict(X_test_vect)
    
    auc_score=roc_auc_score(y_test,pre)
    return auc_score

In [8]:
MNB()

0.9720812182741116

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def Tfid_vect():
    tf = TfidfVectorizer().fit(X_train)
    feature_names = np.array(tf.get_feature_names())
    X_train_tf = tf.transform(X_train)
    max_tf_idfs = X_train_tf.max(0).toarray()[0]
    sorted_tf_idxs = max_tf_idfs.argsort()
    sorted_tf_idfs = max_tf_idfs[sorted_tf_idxs]
    smallest_tf_idfs = pd.Series(sorted_tf_idfs[:20], index=feature_names[sorted_tf_idxs[:20]])                    
    largest_tf_idfs = pd.Series(sorted_tf_idfs[-20:][::-1], index=feature_names[sorted_tf_idxs[-20:][::-1]])
    return (smallest_tf_idfs, largest_tf_idfs)

In [10]:
Tfid_vect()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dty

In [11]:
def Tfid_model():
    vect=TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    
    clf=MultinomialNB(alpha=.1)
    model=clf.fit(X_train_vectorized,y_train)
    
    predictons=model.predict(X_test_vectorized)
    auc_score=roc_auc_score(y_test,predictons)
    return auc_score

In [12]:
Tfid_model()

0.9416243654822335

In [13]:
spam_data.head()

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [14]:
def analysis_1():
    spam_df=spam_data[spam_data['target']==1]
    nonspam_df=spam_data[spam_data['target']==0]

    spam_df['no._of_char']=[len(s) for s in spam_df.loc[:,'text']]
    nonspam_df['no._of_char']=[len(s) for s in nonspam_df.loc[:,'text']]
    
    return (((nonspam_df['no._of_char'].sum())/(nonspam_df.shape[0])),((spam_df['no._of_char'].sum())/(spam_df.shape[0])))

In [15]:
analysis_1()

C:\Users\coolbuddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\coolbuddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [16]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Tuning Model and adding more features

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.


In [17]:
from sklearn.svm import SVC

def SVC_Model():
    vect=TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    
    X_train_final=add_feature(X_train_vectorized,[len(s) for s in X_train])
    X_test_final=add_feature(X_test_vectorized,[len(s) for s in X_test])
    
    clf=SVC(C=10000)
    model=clf.fit(X_train_final,y_train)
    
    predictions=model.predict(X_test_final)
    auc_score=roc_auc_score(y_test,predictions)
    
    
    return auc_score

In [18]:
SVC_Model()

C:\Users\coolbuddy\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9581366823421557

In [19]:
def analysis_2():
    digit_spam=[sum(char.isnumeric() for char in x)for x in spam_data.loc[spam_data['target']==1,'text']]
    digit_nonspam=[sum(char.isnumeric() for char in x) for x in spam_data.loc[spam_data['target']==0,'text']]
    return (np.mean(digit_nonspam),np.mean(digit_spam))

In [20]:
analysis_2()

(0.2992746113989637, 15.76037483266399)

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).
*

In [21]:
from sklearn.linear_model import LogisticRegression

def Log_model():
    
    dig_docs_train = [sum(char.isnumeric() for char in x) for x in X_train]
    dig_docs_test = [sum(char.isnumeric() for char in x) for x in X_test]
    tf = TfidfVectorizer(min_df=5, ngram_range=(1, 3)).fit(X_train)
    X_train_tf = tf.transform(X_train)
    X_test_tf = tf.transform(X_test)
    X_train_tf = add_feature(X_train_tf, dig_docs_train)
    X_test_tf = add_feature(X_test_tf, dig_docs_test)
    clf = LogisticRegression(C=100)
    clf.fit(X_train_tf, y_train)
    preds_test = clf.predict(X_test_tf)
    return roc_auc_score(y_test, preds_test)

In [22]:
Log_model()

C:\Users\coolbuddy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9678709064054463

In [23]:
def analysis_3():
    
    
    return (np.mean(spam_data.loc[spam_data['target']==0,'text'].str.count('\W')), 
            np.mean(spam_data.loc[spam_data['target']==1,'text'].str.count('\W')))

In [24]:
analysis_3()

(17.29181347150259, 29.041499330655956)

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**
`analyzer='char_wb'`

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. 

In [25]:
def Final_model():
    vectorizer=CountVectorizer(min_df=5,ngram_range=(1,3),analyzer='char_wb').fit(X_train)
    X_train_v=vectorizer.transform(X_train)
    X_test_v=vectorizer.transform(X_test)
    

    
    xtr3=[X_train.str.count('\W')]
    xt3=[X_test.str.count('\W')]
    X_train_v=add_feature(X_train_v,xtr3)
    X_test_v=add_feature(X_test_v,xt3)
    
    clf=LogisticRegression(C=100)
    model=clf.fit(X_train_v,y_train)
    predictions=model.predict(X_test_v)
    
    auc_score=roc_auc_score(y_test,predictions)
    return auc_score

In [26]:
Final_model()

C:\Users\coolbuddy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9784412508700744